In [1]:
# univariate multi-step encoder-decoder convlstm for the power usage dataset
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from pandas import DataFrame
import pandas as pd
from keras import optimizers
import keras.backend as K
import numpy as np

# split a univariate dataset into train/test sets
def split_dataset(data,n):
	# split into standard weeks
	train, test = data[0:-n], data[-n:]    
	# restructure into windows of weekly data
	#train = array(split(train, len(train)/7))
	#test = array(split(test, len(test)/7))
	train = array(split(train, len(train)/4))
	test = array(split(test, len(test)/4))    
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	mae_scores = list()      
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		mae = mean_absolute_error(actual[:, i], predicted[:, i])            
		# store
		scores.append(mae)
		mae_scores.append(mae)        
	# calculate overall RMSE
	s = 0
	rmse_=DataFrame(scores)           
	mae_=DataFrame(mae_scores)    
	sonuc=pd.concat([rmse_, mae_], axis=1, ignore_index=True)        
	export_sonuc= sonuc.to_excel (r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\MULTISTEP\LSTM\encoder_decoder_lstm_sonuc.xlsx', index = None, header=True)        
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=4):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end < len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train,test, n_steps, n_length, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	test_x, test_y = to_supervised(test, n_input)  
	# define parameters
	verbose, epochs, batch_size = 0, 20, 16
	n_features, n_outputs = train_x.shape[2], train_y.shape[1]
	# reshape into subsequences [samples, timesteps, rows, cols, channels]
	train_x = train_x.reshape((train_x.shape[0], n_steps, 1, n_length, n_features))
	test_x = test_x.reshape((test_x.shape[0], n_steps, 1, n_length, n_features))    
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))    
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Flatten())
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	plot_model(model, show_shapes=True, to_file='convlstm_encoder_decoder_lstm.png')     
	initial_weights = model.get_weights()    
	# fit network
	repeats=1
	for i in range(repeats):
		#model.set_weights(initial_weights)      
		model.compile(loss='mse', optimizer='adam', metrics=['mse','accuracy','mape'])          
		aa=model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,validation_data=(test_x, test_y), verbose=verbose) 
		pyplot.plot(aa.history['loss'],  color='blue')
		pyplot.plot(aa.history['val_loss'], color='orange')
		#print('%d) TrainRMSE=%f, TestRMSE=%f' % (i, xx.history['loss'].iloc[-1], xx.history['val_loss'].iloc[-1]))
		model.reset_states() 
		#reset_weights(model)         
	pyplot.savefig('epochs_diagnostic_convlstm_lstm.png')   
	pyplot.legend()
	pyplot.show()     
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_steps, n_length, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 0]
	# reshape into [samples, timesteps, rows, cols, channels]
	input_x = input_x.reshape((1, n_steps, 1, n_length, 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_steps, n_length, n_input):
	# fit model
	model = build_model(train,test, n_steps, n_length, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_steps, n_length, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	print(predictions.shape) 
	pred=pd.DataFrame(predictions.reshape(predictions.shape[0],predictions.shape[1]))   
	test_=pd.DataFrame(test[:,:,0])    
	pred=pd.concat([pred, test_], axis=1, ignore_index=True)
	export_pred = pred.to_excel (r'C:\Users\sezgi.sener\Desktop\New folder\deep_learning_time_series_forecasting\code\MULTISTEP\LSTM\convlstm_encoder_lstm_predictions.xlsx', index = None, header=True)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

# load the new file
dataset = read_csv('MyData_weeks.csv', header=0, infer_datetime_format=True, 
                   parse_dates=['Weeks'], index_col=['Weeks'])# split into train and test
# split into train and test
son = list()
for n in [8,9,10,11,12,13]:
	#print(n)    
	train, test = split_dataset(dataset.values,n*4)
	# define the names and functions for the models we wish to evaluate
	n_steps, n_length = 2, 4
	# define the total days to use as input
	n_input = n_length * n_steps
	score, scores = evaluate_model(train, test, n_steps, n_length, n_input)
	# summarize scores
	summarize_scores('lstm', score, scores)
	# evaluate each model
	print(score,scores)
	son.append(scores)
weeks = ['1', '2', '3', '4']
a=0
for i in [8,9,10,11,12,13]:
	pyplot.plot(weeks, DataFrame(son).T[a], marker='o', label=i*4)   
	a=a+1    
pyplot.savefig('multistep_convlstm.png',show_shapes=True, show_layer_names=True) 
pyplot.legend()
pyplot.show()
print(DataFrame(son))
pyplot.boxplot(DataFrame(son).T)
pyplot.show()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


No handles with labels found to put in legend.


<Figure size 640x480 with 1 Axes>

(8, 4, 1)
lstm: [45.108] 14.5, 46.9, 37.1, 33.6
45.10769042360252 [14.494435920478217, 46.85155129100309, 37.14068625701276, 33.62861532213623]


...

<Figure size 640x480 with 1 Axes>

(13, 4, 1)
lstm: [35.533] 13.5, 32.5, 27.8, 26.8
35.53311203344354 [13.548216401974553, 32.46950667151659, 27.839304477891957, 26.831696611762496]


<Figure size 640x480 with 1 Axes>

           0          1          2          3
0  14.494436  46.851551  37.140686  33.628615
1   9.508791  31.349778  26.716490  21.645018
2  12.049005  37.205303  31.874851  28.690368
3  14.253980  32.370871  31.693747  26.105658
4  17.306543  34.078037  31.249390  27.616929
5  13.548216  32.469507  27.839304  26.831697


<Figure size 640x480 with 1 Axes>